In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import os

# Setup Chrome options
chrome_options = Options()
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--window-size=1920x1080")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36')
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])

# Path to ChromeDriver
chromedriver_path = r"C:\Users\yujit\OneDrive\Desktop\chromedriver-win64\chromedriver.exe"
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Open the target webpage
url = "https://dataportalforcities.org/north-america"
driver.get(url)
wait = WebDriverWait(driver, 10)

# Output file
output_file = "city_data.xlsx"

# Load existing data if file exists
if os.path.exists(output_file):
    df_existing = pd.read_excel(output_file)
    scraped_data = set(df_existing.apply(tuple, axis=1))  # Convert rows to set
    print(f"✅ Resuming from last stopped row. {len(scraped_data)} rows already collected.")
else:
    df_existing = pd.DataFrame(columns=["City", "Country", "Population", "Total GHGs (Ton CO2e)"])
    scraped_data = set()

# Function to extract data from the table rows
def extract_data():
    rows = driver.find_elements(By.XPATH, "//table/tbody/tr")
    data = []
    
    for row in rows:
        try:
            columns = row.find_elements(By.TAG_NAME, "td")
            if len(columns) < 4:
                continue  # Skip if there are not enough columns

            city = columns[0].text.strip()
            country = columns[1].text.strip()
            population = columns[2].text.strip()
            ghg = columns[3].find_element(By.CLASS_NAME, "number-format").text.strip()

            row_tuple = (city, country, population, ghg)
            if row_tuple not in scraped_data:  # Avoid duplicate entries
                scraped_data.add(row_tuple)
                data.append([city, country, population, ghg])
        except Exception as e:
            print(f"Error extracting row: {e}")
    
    return data

# Function to append data to Excel in real-time
def append_to_excel(data):
    if data:
        df = pd.DataFrame(data, columns=["City", "Country", "Population", "Total GHGs (Ton CO2e)"])

        while True:
            try:
                with pd.ExcelWriter(output_file, mode="a", if_sheet_exists="overlay", engine="openpyxl") as writer:
                    df.to_excel(writer, index=False, header=False, startrow=writer.sheets["Sheet1"].max_row)
                break  # Exit loop if writing is successful

            except PermissionError:
                print("⚠️ Excel file is open! Please close it to continue saving data.")
                time.sleep(2)  # Wait for 2 seconds before retrying

# Function to properly scroll and locate the "Load More" button
def scroll_and_find_load_more():
    scroll_attempts = 0
    max_scroll_attempts = 15  # To prevent infinite loops

    while scroll_attempts < max_scroll_attempts:
        try:
            driver.execute_script("window.scrollBy(0, 300);")  # Scroll in small steps
            time.sleep(1)  # Short delay for page load

            # Try finding the Load More button
            load_more_button = driver.find_element(By.CLASS_NAME, "region-table-load-more")
            
            if load_more_button.is_displayed():
                print("✅ Load More button is now visible!")
                return load_more_button  # Button is in view, return it

        except Exception as e:
            pass  # Continue scrolling if not found

        scroll_attempts += 1

    print("❌ Could not find Load More button. Stopping script.")
    return None  # Return None if scrolling fails

# Function to force-click the "Load More" button
def force_click_load_more():
    retries = 0
    max_retries = 3

    while retries < max_retries:
        try:
            load_more_button = scroll_and_find_load_more()  # Scroll until button is visible

            if load_more_button:
                # Force-click the button using JavaScript
                driver.execute_script("arguments[0].click();", load_more_button)
                print("✅ Load More button clicked!")
                time.sleep(5)  # Wait for data to load
                return True  # Click successful

        except Exception as e:
            print(f"⚠️ Load More click attempt {retries+1} failed: {e}")

        retries += 1
        time.sleep(2)  # Short delay before retrying

    print("❌ Load More button could not be clicked after multiple retries. Stopping script.")
    return False

# Function to ensure scrolling after loading new data
def scroll_after_loading():
    for _ in range(10):
        driver.execute_script("window.scrollBy(0, 300);")
        time.sleep(1)  # Short delay
        new_rows = extract_data()
        if len(new_rows) > 0:
            return True  # Stop scrolling if new data appears
    return False

# Retry count for "Load More" button
retry_count = 0
max_retries = 5

# Keep loading data until all rows are available
while True:
    old_count = len(scraped_data)
    
    # Extract and save data
    new_data = extract_data()
    append_to_excel(new_data)
    
    # Try clicking the "Load More" button
    if not force_click_load_more():
        retry_count += 1
        print(f"⚠️ Load More button issue: Waiting 1 minute before retrying... ({retry_count}/{max_retries})")
        time.sleep(60)  # WAIT FOR 1 MINUTE TO LET DATA LOAD PROPERLY

        if retry_count >= max_retries:
            print("❌ Load More button not working after multiple retries. Stopping script.")
            break  # Exit loop when no more data is available

    # Ensure the page scrolls down and loads new data
    if not scroll_after_loading():
        print("⚠️ No new rows appeared after clicking Load More. Retrying...")
        continue  # Retry the process if no new data appears

    retry_count = 0  # Reset retry counter if new data loaded

    # Stop when no new rows are loaded
    if len(scraped_data) == old_count:
        print("❌ No new rows loaded. Stopping script.")
        break  

print(f"✅ Data scraping completed. {len(scraped_data)} rows collected. Check city_data.xlsx")
driver.quit()


✅ Resuming from last stopped row. 846 rows already collected.
✅ Load More button is now visible!
✅ Load More button clicked!
⚠️ No new rows appeared after clicking Load More. Retrying...
✅ Load More button is now visible!
✅ Load More button clicked!
⚠️ No new rows appeared after clicking Load More. Retrying...
✅ Load More button is now visible!
✅ Load More button clicked!
⚠️ No new rows appeared after clicking Load More. Retrying...
✅ Load More button is now visible!
✅ Load More button clicked!
⚠️ No new rows appeared after clicking Load More. Retrying...
✅ Load More button is now visible!
✅ Load More button clicked!
⚠️ No new rows appeared after clicking Load More. Retrying...
✅ Load More button is now visible!
✅ Load More button clicked!
⚠️ No new rows appeared after clicking Load More. Retrying...
✅ Load More button is now visible!
✅ Load More button clicked!
⚠️ No new rows appeared after clicking Load More. Retrying...
✅ Load More button is now visible!
✅ Load More button clicked!


KeyboardInterrupt: 